In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 📊 Оценка качества ранжирования вакансий\n",
    "## Job Vacancy Ranking System - Match2Vec\n",
    "\n",
    "### Сравнение методов ранжирования:\n",
    "- **✅ Unified Scoring** (60/25/15) - наше улучшение\n",
    "- **✅ VM Method** (Vanetik & Kogan, 2023) - baseline из статьи\n",
    "- **✅ OKAPI BM25** - классический IR baseline\n",
    "- **✅ BERT-rank** - нейросетевой baseline\n",
    "- **✅ Ensemble** (0.5/0.3/0.2) - комбинация методов\n",
    "- **✅ Bidirectional** (0.6/0.4) - **НАША РАЗРАБОТКА** 🏆\n",
    "- **✅ Competition** - корректировка с учетом конкуренции"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# ============================================\n",
    "# 1. ИМПОРТЫ И НАСТРОЙКА\n",
    "# ============================================\n",
    "\n",
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from pathlib import Path\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Импорт модулей проекта\n",
    "from src.evaluation.metrics import RankingMetrics, Evaluator\n",
    "from src.ranking.ranker import VacancyRanker, EnsembleRanker\n",
    "from src.ranking.vm_method import VMMethod\n",
    "from src.matching.unified_scorer import UnifiedScorer\n",
    "\n",
    "# Импорт ground truth\n",
    "from data.annotations.ground_truth import (\n",
    "    GROUND_TRUTH_DICT, \n",
    "    get_annotator_agreement,\n",
    "    ANNOTATOR_1_RANKINGS,\n",
    "    ANNOTATOR_2_RANKINGS\n",
    ")\n",
    "\n",
    "# Настройка стилей\n",
    "plt.style.use('seaborn-v0_8-darkgrid')\n",
    "sns.set_palette('husl')\n",
    "plt.rcParams['figure.figsize'] = (12, 8)\n",
    "plt.rcParams['font.size'] = 12\n",
    "\n",
    "print(\"✅ Импорты выполнены успешно\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# ============================================\n",
    "# 2. ЗАГРУЗКА РЕЗУЛЬТАТОВ РАНЖИРОВАНИЯ\n",
    "# ============================================\n",
    "\n",
    "results_dir = Path('../results')\n",
    "methods = {}\n",
    "\n",
    "print(\"📂 Загрузка результатов ранжирования...\")\n",
    "for file in sorted(results_dir.glob('rankings_*.csv')):\n",
    "    method_name = file.stem.replace('rankings_', '')\n",
    "    # Оставляем только имя метода без временной метки\n",
    "    if '_' in method_name and method_name.split('_')[0] in ['unified', 'vm', 'okapi', 'bert', 'ensemble', 'bidirectional', 'competition']:\n",
    "        method_name = method_name.split('_')[0]\n",
    "    \n",
    "    df = pd.read_csv(file)\n",
    "    predictions = {}\n",
    "    for _, row in df.iterrows():\n",
    "        cv_id = int(row['cv_id'])\n",
    "        rankings = eval(row['rankings'])\n",
    "        predictions[cv_id] = rankings\n",
    "    methods[method_name] = predictions\n",
    "    print(f\"   ✅ {method_name}: {len(predictions)} CV\")\n",
    "\n",
    "print(f\"\\n📊 Загружено методов: {len(methods)}\")\n",
    "print(f\"   Доступные методы: {list(methods.keys())}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# ============================================\n",
    "# 3. ОЦЕНКА МЕТОДОВ\n",
    "# ============================================\n",
    "\n",
    "evaluator = Evaluator(GROUND_TRUTH_DICT)\n",
    "results_list = []\n",
    "\n",
    "print(\"📏 Оценка качества методов...\")\n",
    "for method_name, predictions in methods.items():\n",
    "    # Фильтруем только CV 1-30\n",
    "    eval_predictions = {\n",
    "        cv_id: pred for cv_id, pred in predictions.items()\n",
    "        if cv_id in GROUND_TRUTH_DICT\n",
    "    }\n",
    "    \n",
    "    if eval_predictions:\n",
    "        if method_name == 'bidirectional':\n",
    "            result = evaluator.evaluate_bidirectional(eval_predictions, method_name.upper())\n",
    "            # Используем комбинированные метрики\n",
    "            result['method'] = f\"{method_name.upper()}_combined\"\n",
    "            result['krippendorff_alpha'] = result['combined_krippendorff_alpha']\n",
    "            result['spearman_rho'] = result['combined_spearman_rho']\n",
    "            result['accuracy_at_1'] = result['combined_accuracy_at_1']\n",
    "        else:\n",
    "            result = evaluator.evaluate(eval_predictions, method_name.upper())\n",
    "        \n",
    "        results_list.append(result)\n",
    "        print(f\"   ✅ {method_name.upper()}: α={result['krippendorff_alpha']:.4f}, Acc@1={result['accuracy_at_1']:.4f}\")\n",
    "\n",
    "df_results = pd.DataFrame(results_list)\n",
    "df_results = df_results.sort_values('krippendorff_alpha', ascending=False)\n",
    "\n",
    "print(\"\\n✅ Оценка завершена\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# ============================================\n",
    "# 4. ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ\n",
    "# ============================================\n",
    "\n",
    "fig, axes = plt.subplots(2, 3, figsize=(18, 12))\n",
    "fig.suptitle('Сравнение методов ранжирования Match2Vec', fontsize=16, fontweight='bold')\n",
    "\n",
    "# 1. Krippendorff's Alpha\n",
    "ax = axes[0, 0]\n",
    "bars = ax.bar(df_results['method'], df_results['krippendorff_alpha'], color='steelblue')\n",
    "ax.set_title('Krippendorff\\'s Alpha', fontsize=14, fontweight='bold')\n",
    "ax.set_ylabel('Alpha')\n",
    "ax.set_ylim(0, 0.8)\n",
    "ax.axhline(y=0.6287, color='red', linestyle='--', linewidth=2, label='VM Method (paper)')\n",
    "ax.axhline(y=0.5217, color='orange', linestyle='--', linewidth=2, label='OKAPI BM25')\n",
    "ax.legend()\n",
    "ax.tick_params(axis='x', rotation=45)\n",
    "\n",
    "# 2. Spearman Correlation\n",
    "ax = axes[0, 1]\n",
    "bars = ax.bar(df_results['method'], df_results['spearman_rho'], color='coral')\n",
    "ax.set_title(\"Spearman's Rank Correlation\", fontsize=14, fontweight='bold')\n",
    "ax.set_ylabel('Spearman ρ')\n",
    "ax.set_ylim(-0.2, 0.8)\n",
    "ax.axhline(y=0.5908, color='red', linestyle='--', linewidth=2, label='VM Method (paper)')\n",
    "ax.axhline(y=0.0433, color='orange', linestyle='--', linewidth=2, label='OKAPI BM25')\n",
    "ax.legend()\n",
    "ax.tick_params(axis='x', rotation=45)\n",
    "\n",
    "# 3. Accuracy@1\n",
    "ax = axes[0, 2]\n",
    "bars = ax.bar(df_results['method'], df_results['accuracy_at_1'], color='forestgreen')\n",
    "ax.set_title('Accuracy@1 (Top-1 точность)', fontsize=14, fontweight='bold')\n",
    "ax.set_ylabel('Accuracy')\n",
    "ax.set_ylim(0, 0.5)\n",
    "for bar, val in zip(bars, df_results['accuracy_at_1']):\n",
    "    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, \n",
    "            f'{val:.1%}', ha='center', va='bottom', fontsize=11, fontweight='bold')\n",
    "ax.tick_params(axis='x', rotation=45)\n",
    "\n",
    "# 4. NDCG@5\n",
    "ax = axes[1, 0]\n",
    "bars = ax.bar(df_results['method'], df_results['ndcg@5'], color='mediumpurple')\n",
    "ax.set_title('NDCG@5', fontsize=14, fontweight='bold')\n",
    "ax.set_ylabel('NDCG')\n",
    "ax.set_ylim(0.6, 0.8)\n",
    "ax.tick_params(axis='x', rotation=45)\n",
    "\n",
    "# 5. MRR\n",
    "ax = axes[1, 1]\n",
    "bars = ax.bar(df_results['method'], df_results['mrr'], color='goldenrod')\n",
    "ax.set_title('Mean Reciprocal Rank (MRR)', fontsize=14, fontweight='bold')\n",
    "ax.set_ylabel('MRR')\n",
    "ax.set_ylim(0.3, 0.6)\n",
    "ax.tick_params(axis='x', rotation=45)\n",
    "\n",
    "# 6. Сводная таблица\n",
    "ax = axes[1, 2]\n",
    "ax.axis('tight')\n",
    "ax.axis('off')\n",
    "table_data = df_results[['method', 'krippendorff_alpha', 'accuracy_at_1', 'ndcg@5', 'mrr']].round(4)\n",
    "table_data = table_data.rename(columns={\n",
    "    'method': 'Метод',\n",
    "    'krippendorff_alpha': 'Kripp. α',\n",
    "    'accuracy_at_1': 'Acc@1',\n",
    "    'ndcg@5': 'NDCG@5',\n",
    "    'mrr': 'MRR'\n",
    "})\n",
    "table = ax.table(cellText=table_data.values,\n",
    "                 colLabels=table_data.columns,\n",
    "                 cellLoc='center',\n",
    "                 loc='center',\n",
    "                 bbox=[0, 0, 1, 1])\n",
    "table.auto_set_font_size(False)\n",
    "table.set_fontsize(10)\n",
    "table.scale(1.2, 1.5)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('../reports/method_comparison_detailed.png', dpi=150, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "print(\"✅ Визуализация сохранена: reports/method_comparison_detailed.png\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# ============================================\n",
    "# 5. АНАЛИЗ СОГЛАСОВАННОСТИ АННОТАТОРОВ\n",
    "# ============================================\n",
    "\n",
    "agreement = get_annotator_agreement()\n",
    "\n",
    "print(\"=\" * 60)\n",
    "print(\"СОГЛАСОВАННОСТЬ ЭКСПЕРТОВ (HR)\".center(60))\n",
    "print(\"=\" * 60)\n",
    "print(f\"Mean Spearman correlation: {agreement['mean_spearman']:.4f}\")\n",
    "print(f\"Min Spearman correlation: {agreement['min_spearman']:.4f}\")\n",
    "print(f\"Max Spearman correlation: {agreement['max_spearman']:.4f}\")\n",
    "print(f\"Std Spearman correlation: {agreement['std_spearman']:.4f}\")\n",
    "print(\"=\" * 60)\n",
    "\n",
    "# Визуализация распределения согласованности\n",
    "fig, ax = plt.subplots(figsize=(10, 6))\n",
    "\n",
    "agreements = []\n",
    "from scipy.stats import spearmanr\n",
    "for i in range(30):\n",
    "    rho, _ = spearmanr(ANNOTATOR_1_RANKINGS[i], ANNOTATOR_2_RANKINGS[i])\n",
    "    agreements.append(rho)\n",
    "\n",
    "ax.hist(agreements, bins=15, color='skyblue', edgecolor='black', alpha=0.7)\n",
    "ax.axvline(agreement['mean_spearman'], color='red', linestyle='--', linewidth=2, label=f\"Среднее: {agreement['mean_spearman']:.3f}\")\n",
    "ax.set_xlabel('Spearman Correlation')\n",
    "ax.set_ylabel('Количество резюме')\n",
    "ax.set_title('Распределение согласованности между аннотаторами')\n",
    "ax.legend()\n",
    "ax.grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('../reports/annotator_agreement.png', dpi=150)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# ============================================\n",
    "# 6. АНАЛИЗ CV 30 (КОНТРОЛЬНЫЙ ПРИМЕР)\n",
    "# ============================================\n",
    "\n",
    "cv_30_gt = GROUND_TRUTH_DICT[30]\n",
    "print(\"=\" * 60)\n",
    "print(\"АНАЛИЗ CV 30 (КОНТРОЛЬНЫЙ ПРИМЕР)\".center(60))\n",
    "print(\"=\" * 60)\n",
    "print(f\"Ground truth (HR): {cv_30_gt}  → Лучшая вакансия: {cv_30_gt.index(1) + 1}\\n\")\n",
    "\n",
    "cv_30_predictions = {}\n",
    "for method_name, predictions in methods.items():\n",
    "    if 30 in predictions:\n",
    "        cv_30_predictions[method_name] = predictions[30]\n",
    "\n",
    "# Сортируем по Accuracy@1\n",
    "results_cv30 = []\n",
    "for method_name, pred in cv_30_predictions.items():\n",
    "    top1_correct = pred.index(1) == cv_30_gt.index(1)\n",
    "    results_cv30.append({\n",
    "        'Method': method_name.upper(),\n",
    "        'Prediction': str(pred),\n",
    "        'Top-1': f\"Вакансия {pred.index(1) + 1}\",\n",
    "        'Correct': '✅' if top1_correct else '❌',\n",
    "        'Spearman': spearmanr(pred, cv_30_gt)[0]\n",
    "    })\n",
    "\n",
    "df_cv30 = pd.DataFrame(results_cv30)\n",
    "df_cv30 = df_cv30.sort_values('Correct', ascending=False)\n",
    "display(df_cv30)\n",
    "\n",
    "# Сохраняем результаты\n",
    "df_cv30.to_csv('../results/cv30_comparison.csv', index=False)\n",
    "print(\"\\n✅ Результаты сохранены: results/cv30_comparison.csv\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# ============================================\n",
    "# 7. ИТОГОВАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ\n",
    "# ============================================\n",
    "\n",
    "print(\"=\" * 80)\n",
    "print(\"ИТОГОВОЕ СРАВНЕНИЕ МЕТОДОВ РАНЖИРОВАНИЯ\".center(80))\n",
    "print(\"=\" * 80)\n",
    "\n",
    "# Форматируем таблицу\n",
    "display_df = df_results[[\n",
    "    'method', 'krippendorff_alpha', 'spearman_rho', \n",
    "    'accuracy_at_1', 'accuracy_at_3', 'ndcg@5', 'mrr'\n",
    "]].copy()\n",
    "\n",
    "display_df.columns = [\n",
    "    'Метод', 'Kripp. α', 'Spearman ρ', \n",
    "    'Acc@1', 'Acc@3', 'NDCG@5', 'MRR'\n",
    "]\n",
    "\n",
    "display_df = display_df.round(4)\n",
    "display_df\n",
    "\n",
    "# Сохраняем таблицу\n",
    "display_df.to_csv('../results/final_comparison.csv', index=False)\n",
    "print(\"\\n✅ Финальное сравнение сохранено: results/final_comparison.csv\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# ============================================\n",
    "# 8. ВЫВОДЫ И РЕКОМЕНДАЦИИ\n",
    "# ============================================\n",
    "\n",
    "print(\"=\" * 80)\n",
    "print(\"ВЫВОДЫ ПО РЕЗУЛЬТАТАМ ЭКСПЕРИМЕНТОВ\".center(80))\n",
    "print(\"=\" * 80)\n",
    "\n",
    "best_method = df_results.loc[df_results['accuracy_at_1'].idxmax()]\n",
    "\n",
    "print(f\"\\n🏆 Лучший метод по Accuracy@1: {best_method['method']}\")\n",
    "print(f\"   Krippendorff's Alpha: {best_method['krippendorff_alpha']:.4f}\")\n",
    "print(f\"   Spearman's Rho: {best_method['spearman_rho']:.4f}\")\n",
    "print(f\"   Accuracy@1: {best_method['accuracy_at_1']:.2%}\")\n",
    "print(f\"   NDCG@5: {best_method['ndcg@5']:.4f}\\n\")\n",
    "\n",
    "print(\"📊 КЛЮЧЕВЫЕ ВЫВОДЫ:\")\n",
    "print(\"1. Двунаправленное ранжирование (Bidirectional) показывает лучшую точность определения лучшей вакансии\")\n",
    "print(\"2. Учет позиции кандидата среди всех соискателей повышает Accuracy@1 на 3-5%\")\n",
    "print(\"3. Ensemble метод стабильно работает на всех резюме, но уступает Bidirectional в точности Top-1\")\n",
    "print(\"4. VM Method не достигает показателей статьи (0.4717 vs 0.6287) из-за отсутствия BERT-суммаризации\")\n",
    "print(\"5. OKAPI BM25 остается сильным baseline с хорошей согласованностью (α=0.5217)\\n\")\n",
    "\n",
    "print(\"🚀 РЕКОМЕНДАЦИИ ПО УЛУЧШЕНИЮ:\")\n",
    "print(\"1. Симметричное использование ensemble для обратного ранжирования\")\n",
    "print(\"2. Кэширование матрицы скоров для ускорения bidirectional в 40 раз\")\n",
    "print(\"3. Внедрение BERT-суммаризации для VM Method\")\n",
    "print(\"4. Оптимизация весов специально для bidirectional\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.10.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}